## *Exercise 1: Web Scraping, Portfolio Analysis & Visualization*
### Objective: Scrape top stock gainers from Yahoo Finance, analyze their historical performance, construct a simulated portfolio, and visualize its risk and return.

### 1. Environment Setup:

In [1]:
%pip install webdriver-manager selenium pandas yfinance matplotlib seaborn beautifulsoup4 #%pip → es un magic command de Jupyter/VS Code Notebooks para instalar paquetes dentro del entorno del kernel activo.

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#%pip': Expected package name at the start of dependency specifier
    #%pip
    ^


In [2]:
# Herramientas de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By # Indica el submódulo exacto dentro del paquete Selenium más rapido
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import yfinance as yf

C:\Users\diego\AppData\Local\Temp\ipykernel_20364\2616715651.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 2. Scrape Top 50 Gainers:

In [1]:
# Configuramos las opciones de Chrome para que no tenga probelmas.
chrome_options = Options()

# chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized") # Pantalla completa
chrome_options.add_argument("--lang=en-US") # Lenguaje inglés

# Iniciar el WebDriver de Chrome
# pasando nuestras 'chrome_options' como argumento.
driver =  webdriver.Chrome(options=chrome_options)

print("WebDriver iniciado con éxito.")

NameError: name 'Options' is not defined

In [4]:
url = "https://finance.yahoo.com/markets/stocks/gainers" 
driver.get(url) # Abrir con , get() la URL en el navegador.
wait = WebDriverWait(driver, 15)
# 1) Localizar el botón "Rows per page" para poner 50 filas
btn = wait.until(EC.element_to_be_clickable(
    (By.XPATH, "//button[@aria-haspopup='listbox']"))
)
btn.click()

# 2) Esperar a que el menú esté visible
menu_container = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.dialog-container[aria-hidden='false']"))
)

# 3) Seleccionar la opción "50"
opt_50 = menu_container.find_element(By.CSS_SELECTOR, "[role='option'][data-value='50']")
driver.execute_script("arguments[0].click();", opt_50)

In [5]:
# 1) Descargar el contenedor de la tabla
table_container = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//*[@id='main-content-wrapper']/section[1]/div/div[2]/div")
    )
)

# 2) Dentro del contenedor, buscar todas las filas del <tbody>
rows = table_container.find_elements(By.XPATH, ".//table/tbody/tr")

data = []
for r in rows:
    try:
        symbol = r.find_element(By.XPATH, "./td[1]//a").text.strip()
    except:
        symbol = r.find_element(By.XPATH, "./td[1]").text.strip()
    try:
        name = r.find_element(By.XPATH, "./td[2]").text.strip()
    except:
        name = ""

    data.append((symbol, name))

# 3) Convertir a DataFrame
df = pd.DataFrame(data, columns=["Symbol", "Name"])


In [6]:
df.to_csv("../output/top_50_gainers.csv", index=False) # Lo mandamos a nuestra carpeta output

In [8]:
df.head(50)

,Symbol,Name
0,AMBA,"Ambarella, Inc."
1,IREN,IREN Limited
2,BABA,Alibaba Group Holding Limited
3,AFRM,"Affirm Holdings, Inc."
4,DOOO,BRP Inc.
5,ADSK,"Autodesk, Inc."
6,CIFR,Cipher Mining Inc.
7,SATS,EchoStar Corporation
8,S,"SentinelOne, Inc."
9,EQX,Equinox Gold Corp.


In [9]:
# --- 1) Lista de símbolos ---
tickers = (
    df['Symbol']
      .dropna()
      .astype(str)
      .str.upper()
      .drop_duplicates()
      .tolist()
)

# --- 2) Descargar datos (de julio a julio, ya que falta un dato por agosto) ---
raw = yf.download(
    " ".join(tickers),
    start="2023-07-01",
    end="2024-07-31",
    interval="1mo",
    auto_adjust=False,
    group_by="ticker",
    threads=True,
    progress=False
)

# --- 3) Extraer 'Adj Close' ---
adj_list = []
for t in tickers:
    try:
        s = raw[t]['Adj Close'].rename(t)
        adj_list.append(s)
    except Exception:
        try:
            s = yf.download(
                t,
                start="2023-07-01", end="2024-07-31",
                interval="1mo",
                auto_adjust=False,
                progress=False
            )['Adj Close'].rename(t)
            adj_list.append(s)
        except:
            pass

# --- 4) Concatenar en DataFrame final ---
monthly_adj_close = pd.concat(adj_list, axis=1).sort_index()
monthly_adj_close.index.name = "Date"

print("DataFrame final de julio 2023 a julio 2024:")
print(monthly_adj_close)






2 Failed downloads:
['SNDK', 'MIAX']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2023-07-01 -> 2024-07-31) (Yahoo error = "Data doesn\'t exist for startDate = 1688184000, endDate = 1722398400")')


DataFrame final de julio 2023 a julio 2024:
                 AMBA    IREN       BABA       AFRM       DOOO        ADSK  \
Date                                                                         
2023-07-01  83.419998   6.660  97.020836  19.389999  90.250786  211.990005   
2023-08-01  62.150002   4.860  88.226669  20.809999  74.706497  221.940002   
2023-09-01  53.029999   3.710  82.376549  21.270000  74.256226  206.910004   
2023-10-01  44.990002   3.080  78.387825  17.610001  66.312378  197.630005   
2023-11-01  58.709999   4.935  71.113159  34.410000  60.536274  218.429993   
2023-12-01  61.290001   7.150  73.610863  49.139999  70.303665  243.479996   
2024-01-01  52.560001   3.920  69.460838  40.509998  61.996098  253.809998   
2024-02-01  55.849998   5.900  71.251022  37.520000  64.776596  258.170013   
2024-03-01  50.770000   5.400  69.643715  37.259998  65.926125  260.420013   
2024-04-01  45.970001   4.340  72.040237  31.879999  66.093147  212.850006   
2024-05-01  58.25999

In [10]:
monthly_adj_close.head(12)

,AMBA,IREN,BABA,AFRM,DOOO,ADSK,CIFR,SATS,S,EQX,...,KGC,FSM,BF-B,BTU,HP,WPM,UPWK,EMN,SNDK,TD
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-01,83.419998,6.660,97.020836,19.389999,90.250786,211.990005,3.80,19.430000,16.670000,5.27,...,4.816687,3.55,67.831390,21.705290,40.458939,43.868923,10.43,79.610649,NaN,59.028961
2023-08-01,62.150002,4.860,88.226669,20.809999,74.706497,221.940002,3.19,17.379999,16.629999,5.05,...,4.903736,3.10,63.536686,20.873444,36.139217,42.645714,14.81,79.080414,NaN,55.266926
2023-09-01,53.029999,3.710,82.376549,21.270000,74.256226,206.910004,2.33,16.750000,16.860001,4.23,...,4.438434,2.72,55.427673,25.223322,38.554230,39.820427,11.36,71.368652,NaN,54.587521
2023-10-01,44.990002,3.080,78.387825,17.610001,66.312378,197.630005,3.33,13.860000,15.630000,4.40,...,5.071105,2.81,54.126278,22.894117,36.185741,41.489838,10.45,70.229019,NaN,50.583595
2023-11-01,58.709999,4.935,71.113159,34.410000,60.536274,218.429993,2.77,10.470000,19.090000,5.48,...,5.732977,3.90,56.612850,23.136745,33.131393,48.020199,14.09,78.780930,NaN,55.925022
2023-12-01,61.290001,7.150,73.610863,49.139999,70.303665,243.479996,4.13,16.570000,27.440001,4.89,...,5.918859,3.86,55.239033,23.681175,33.502346,48.605907,14.87,84.410156,NaN,59.253567
2024-01-01,52.560001,3.920,69.460838,40.509998,61.996098,253.809998,3.04,13.390000,26.799999,4.40,...,5.390563,3.03,53.110741,25.998659,37.239208,46.172653,13.71,79.233444,NaN,55.686607
2024-02-01,55.849998,5.900,71.251022,37.520000,64.776596,258.170013,2.96,13.110000,28.170000,4.05,...,4.793787,2.71,58.267029,24.119352,35.509525,40.577160,13.10,83.206970,NaN,55.671078
2024-03-01,50.770000,5.400,69.643715,37.259998,65.926125,260.420013,5.15,14.250000,23.309999,6.02,...,5.997125,3.73,49.937637,23.697300,39.338261,46.428783,12.26,95.042191,NaN,56.023659


In [11]:
monthly_adj_close.isna().sum()

AMBA     0
IREN     0
BABA     0
AFRM     0
DOOO     0
ADSK     0
CIFR     0
SATS     0
S        0
EQX      0
NXE      0
MIAX    13
NG       0
PRVA     0
SSRM     0
HL       0
CELH     0
HMY      0
OS      12
CNXC     0
BIDU     0
CDE      0
JOYY     0
GH       0
BTDR     0
COO      0
GFI      0
GSAT     0
SBSW     0
IAG      0
SOUN     0
HCC      0
NGD      0
BHC      0
BILI     0
AEM      0
SJM      0
BF-A     0
OLN      0
MOH      0
KGC      0
FSM      0
BF-B     0
BTU      0
HP       0
WPM      0
UPWK     0
EMN      0
SNDK    13
TD       0
dtype: int64

In [13]:
import numpy as np
def select_portfolio_top_sharpe(monthly_adj_close: pd.DataFrame, n_assets: int = 10):
    """
    Selecciona n_assets por mayor Sharpe individual usando la PRIMERA MITAD
    """
    # 1) Retornos mensuales y limpieza
    rets = monthly_adj_close.sort_index().pct_change().dropna(how="all")
    # Si hay muy pocos meses, levanta error claro
    if rets.shape[0] < 4:
        raise ValueError("Se requieren al menos 5 meses de datos para una selección razonable.")

    # 2) Dividir en primera mitad (selección) y segunda mitad (validación futura)
    n = rets.shape[0]
    first_half = rets.iloc[:n//2, :].dropna(axis=1, how="any")  # exige datos completos en la 1ra mitad

    # Si por exigir datos completos te quedas con muy pocos, relaja a 'how="all"'
    if first_half.shape[1] < n_assets:
        first_half = rets.iloc[:n//2, :].dropna(axis=1, how="all")

    # 3) Métricas de selección (Sharpe simple, rf≈0)
    mean_ret = first_half.mean()
    vol_ret  = first_half.std()
    sharpe   = mean_ret / vol_ret.replace(0, np.nan)

    # 4) Top-n por Sharpe (descendente)
    ranked = sharpe.sort_values(ascending=False).dropna()
    selected = ranked.head(n_assets).index.tolist()

    # 5) Pesos equiponderados
    if len(selected) == 0:
        raise ValueError("No hay tickers suficientes con datos para calcular Sharpe.")
    w = pd.Series(1.0/len(selected), index=selected, name="weight")

    # 6) Tabla resumen
    table = pd.DataFrame({
        "MeanRet_1H": mean_ret.reindex(selected),
        "Vol_1H": vol_ret.reindex(selected),
        "Sharpe_1H": sharpe.reindex(selected),
        "Weight": w
    }).sort_values("Sharpe_1H", ascending=False)

    return selected, w, table

# ====== USO ======
selected, weights, selection_table = select_portfolio_top_sharpe(monthly_adj_close, n_assets=10)

# Mostrar los tickers seleccionados
print("Tickers seleccionados (Top-10 por Sharpe en 1ra mitad):")
print(selected)


Tickers seleccionados (Top-10 por Sharpe en 1ra mitad):
['NXE', 'MOH', 'HCC', 'S', 'ADSK', 'AFRM', 'GSAT', 'HMY', 'BTU', 'UPWK']


C:\Users\diego\AppData\Local\Temp\ipykernel_20364\3129577337.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = monthly_adj_close.sort_index().pct_change().dropna(how="all")


In [14]:
# --- Retornos mensuales individuales (últimos 6 meses, ya filtrado) ---
last6_ind_rets = (
    monthly_adj_close[selected]
    .sort_index()
    .pct_change()
    .dropna()
    .tail(6)
)


print("=== Retornos mensuales individuales (últimos 6m) ===")
print(last6_ind_rets)


=== Retornos mensuales individuales (últimos 6m) ===
                 NXE       MOH       HCC         S      ADSK      AFRM  \
Date                                                                     
2024-02-01 -0.077225  0.105123 -0.112202  0.051119  0.017178 -0.073809   
2024-03-01  0.102128  0.042954  0.076355 -0.172524  0.008715 -0.006930   
2024-04-01 -0.019305 -0.167295  0.126030 -0.093522 -0.182666 -0.144391   
2024-05-01  0.020997 -0.080444  0.001170 -0.203502 -0.052854 -0.081869   
2024-06-01 -0.102828 -0.054930 -0.081590  0.250743  0.227431  0.032115   
2024-07-01 -0.044413  0.147898  0.101004  0.087886  0.000283 -0.063555   

                GSAT       HMY       BTU      UPWK  
Date                                                
2024-02-01 -0.018868 -0.077544 -0.072285 -0.044493  
2024-03-01 -0.057692  0.430823 -0.017498 -0.064122  
2024-04-01 -0.122449  0.048960 -0.095631 -0.045677  
2024-05-01 -0.155039  0.071275  0.129444 -0.096581  
2024-06-01  0.027523  0.007692 -0.10

In [15]:
# Pesos equiponderados (10% cada uno)
weights = pd.Series(1.0 / last6_ind_rets.shape[1], index=last6_ind_rets.columns)

# Retornos del portafolio = promedio ponderado de retornos individuales
port_rets_6m = last6_ind_rets.mul(weights, axis=1).sum(axis=1)

print("\n=== Retornos mensuales del portafolio (últimos 6m) ===")
print(port_rets_6m)



=== Retornos mensuales del portafolio (últimos 6m) ===
Date
2024-02-01   -0.030301
2024-03-01    0.034221
2024-04-01   -0.069595
2024-05-01   -0.044740
2024-06-01    0.021883
2024-07-01    0.050095
dtype: float64
